In [77]:
import scipy
import os
import scipy.io as sio
import numpy as np

import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [78]:
# Paths to data
camera_path = "/Volumes/willie/EPID&Camera comparisons/Camera/Matlab"
epid_path = "/Volumes/willie/EPID&Camera comparisons/EPID/Matlab"

In [79]:
camera_frame_names = os.listdir(camera_path)
epid_frame_names = os.listdir(epid_path)

camera_frame_paths = []
epid_frame_paths = []

for filename in camera_frame_names:
    camera_frame_paths.append(os.path.join(camera_path, filename))

for filename in epid_frame_names:
    epid_frame_paths.append(os.path.join(epid_path, filename)) 

In [80]:
camera_frame_numbers = [int(name[:-4]) for name in camera_frame_names]
camera_frame_paths_sorted = [c for _,c in sorted(zip(camera_frame_numbers, camera_frame_paths))]


In [81]:
epid_frame_numbers = [int(name[:-4]) for name in epid_frame_names]
epid_frame_paths_sorted = [c for _,c in sorted(zip(epid_frame_numbers, epid_frame_paths))]

In [82]:
camera_frame_paths = camera_frame_paths_sorted
epid_frame_paths = epid_frame_paths_sorted

## Load data 

In [83]:
camera_frames = []
epid_frames = []

for f in camera_frame_paths:
    data = sio.loadmat(f)
    frame = data['tmp']
    frame = np.rot90(frame, 1)
    camera_frames.append(frame)
        
    
for f in epid_frame_paths:
    data = sio.loadmat(f)
    frame = data['New']
    epid_frames.append(frame)
    
# Cut the stray images from the beginning
# epid_frames = epid_frames[len(epid_frames)-len(camera_frames):]

len(epid_frames)

75

### Remove extra frames

In [84]:
extra_frames_epid = [67, 60,56, 51, 45, 46, 43, 38, 37, 34, 28, 27, 22, 19, 16, 12, 11, 8, 7, 6, 5, 4, 3, 2, 1]
extra_frames_camera = [1, 2, 3, 4, 5, 6]


epid_temp = []
camera_temp = []

for i in range(0, len(epid_frames)):
    if i+1 not in extra_frames_epid:
        epid_temp.append(epid_frames[i])
        
for i in range(0, len(camera_frames)):
    if i+1 not in extra_frames_camera:
        camera_temp.append(camera_frames[i])
        
epid_frames = epid_temp
camera_frames = camera_temp


## Augment data

In [85]:
## epid frames
i = 0
new_data_train = []

for frame in epid_frames:
    frame = frame[300:700, 300:700]
    frame = cv2.resize(frame, (312, 312))
    
    frame = (frame - np.min(frame))
    frame = (frame / np.max(frame) * 255).astype('int')
    
    new_data_train.append(frame)
    new_data_train.append(np.flip(frame, 0))
    new_data_train.append(np.flip(frame, 1))
    new_data_train.append(np.rot90(frame, 1))
    new_data_train.append(np.rot90(frame, 2))
    new_data_train.append(np.rot90(frame, 3))

## camera frames
i = 0
new_data_noise = []

for frame in camera_frames:
    frame = (frame - np.min(frame))
    frame = (frame / np.max(frame) * 255).astype('int')
        
    new_data_noise.append(frame)
    new_data_noise.append(np.flip(frame, 0))
    new_data_noise.append(np.flip(frame, 1))
    new_data_noise.append(np.rot90(frame, 1))
    new_data_noise.append(np.rot90(frame, 2))
    new_data_noise.append(np.rot90(frame, 3))

    

In [87]:
i = 0
j = 0
for frame in new_data_train:
    cv2.imwrite('train/frame_{}.jpg'.format(i), frame)
    i = i + 1
    
for frame in new_data_noise:
    cv2.imwrite('noise/frame_{}.jpg'.format(j), frame)
    j = j + 1